In [13]:
# import packages
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import numpy as np

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set X_test, y_test. Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally. Here is a flowchart of typical cross validation workflow in model training. The best parameters can be determined by grid search techniques.


<img src='https://scikit-learn.org/stable/_images/grid_search_workflow.png' width=50% />

In scikit-learn a random split into training and test sets can be quickly computed with the `train_test_split` helper function. Let’s load the iris data set to fit a linear support vector machine on it:

In [4]:
X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

print('Train Data:\t', X_train.shape, y_train.shape)

print('Test Data:\t', X_test.shape, y_test.shape)


clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

Train Data:	 (90, 4) (90,)
Test Data:	 (60, 4) (60,)


0.9666666666666667

When **evaluating different settings (“hyperparameters”)** for estimators, such as the C setting that must be manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can **“leak”** into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called **“validation set”**: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called **cross-validation** (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

* A model is trained using k-1 of the folds as training data;
* the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then **the average of the values computed in the loop**. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage in problems such as inverse inference where the number of samples is very small.

<img src='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png' width=60%/>


# 3.1.1. Computing cross-validated metrics
### Function:  `cross_val_score(clf, X, y, cv=cv)`
The simplest way to use cross-validation is to call the `cross_val_score` helper function on the estimator and the dataset.

The following example demonstrates how to estimate the accuracy of a linear kernel support vector machine on the iris dataset by splitting the data, fitting a model and computing the score 5 consecutive times (with different splits each time):


In [10]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
print(scores)
# The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[0.96666667 1.         0.96666667 0.96666667 1.        ]
0.98 accuracy with a standard deviation of 0.02


By default, the score computed at each CV iteration is the `score` method of the estimator. It is possible to change this by using the scoring parameter:

In [11]:
from sklearn import metrics
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
scores

array([0.96658312, 1.        , 0.96658312, 0.96658312, 1.        ])

Another option is to use an iterable yielding (train, test) splits as arrays of indices, for example:

In [14]:
def custom_cv_2folds(X):
    n = X.shape[0]
    i = 1
    while i <= 2:
        idx = np.arange(n * (i - 1) / 2, n * i / 2, dtype=int)
        yield idx, idx
        i += 1

custom_cv = custom_cv_2folds(X)
cross_val_score(clf, X, y, cv=custom_cv)

array([1.        , 0.97333333])

## Data transformation with held out data
### Function: `preprocessing.StandardScaler().fit(X_train)`

Just as it is important to test a predictor on data held-out from training, preprocessing (such as standardization, feature selection, etc.) and similar data transformations similarly should be learnt from a training set and applied to held-out data for prediction:

In [17]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
test_score_transformed = clf.score(X_test_transformed, y_test)
test_score = clf.score(X_test, y_test)
print(f'test score on transformed:{test_score_transformed}, untransformed:{test_score}')

test score on transformed:0.9333333333333333, untransformed:0.35


### Function: `make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))`
A Pipeline makes it easier to compose estimators, providing this behavior under cross-validation:


In [20]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
cross_val_score(clf, X, y, cv=5)

array([0.96666667, 0.96666667, 0.96666667, 0.93333333, 1.        ])

# 3.1.2. Cross validation iterators
The following sections list utilities to generate indices that can be used to generate dataset splits according to different cross validation strategies.

## 3.1.2.1. Cross-validation iterators for i.i.d. data

### Function: `KFold(n_splits=n_splits)`
KFold divides all the samples in k groups of samples, called folds (if k = n, this is equivalent to the Leave One Out strategy), of equal sizes (if possible). The prediction function is learned using k - 1 folds, and the fold left out is used for test.

Example of 2-fold cross-validation on a dataset with 4 samples:

In [24]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print(f"train index: {train} \t test index: {test}")

train index: [2 3] 	 test index: [0 1]
train index: [0 1] 	 test index: [2 3]


Here is a visualization of the cross-validation behavior. Note that KFold is not affected by classes or groups.
<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png' width=70%/>

### Function: `RepeatedKFold()`

In [28]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
random_state = 12883823
rkf = RepeatedKFold(n_splits=2, n_repeats=4, random_state=random_state)
for train, test in rkf.split(X):
    print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]
[0 2] [1 3]
[1 3] [0 2]
[0 3] [1 2]
[1 2] [0 3]
[1 2] [0 3]
[0 3] [1 2]


### Function: `ShuffleSplit()`
The `ShuffleSplit` iterator will generate a user defined number of independent train / test dataset splits. Samples are first shuffled and then split into a pair of train and test sets.

It is possible to control the randomness for reproducibility of the results by explicitly seeding the `random_state` pseudo random number generator.

In [35]:
from sklearn.model_selection import ShuffleSplit
X = np.arange(10)
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=1)
for train_index, test_index in ss.split(X):
    print("%s %s" % (train_index, test_index))


[4 0 3 1 7 8 5] [2 9 6]
[0 8 4 2 1 6 7] [9 5 3]
[9 0 6 1 7 4 2] [8 3 5]
[0 6 1 5 8 7 9] [2 4 3]
[0 1 2 3 5 9 8] [6 4 7]


Here is a visualization of the cross-validation behavior. Note that ShuffleSplit is not affected by classes or groups.
ShuffleSplit is thus a good alternative to KFold cross validation that allows a finer control on the number of iterations and the proportion of samples on each side of the train / test split.
<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_008.png' width=70%/>

## 3.1.2.2. Cross-validation iterators with stratification based on class labels.

### Function : `StratifiedKFold()`
StratifiedKFold is a variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set. Here is an example of stratified 3-fold cross-validation on a dataset with 50 samples from two unbalanced classes. We show the number of samples in each class and compare with KFold.

We can see that `StratifiedKFold` **preserves the class ratios** (approximately 1 / 10) in both train and test dataset.

In [43]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))
skf = StratifiedKFold(n_splits=3)
print('=== Stratified K-Fold! ===')
for train, test in skf.split(X, y):
    print('train [N neg, N Pos]:  {}   |   test [N neg, N Pos]: {}'.format(np.bincount(y[train]), np.bincount(y[test])))


print('=== UnStratified K-Fold! ===')
kf = KFold(n_splits=3)
for train, test in kf.split(X, y):
    print('train [N neg, N Pos]:   |   test [N neg, N Pos]:  {}'.format(
        np.bincount(y[train]), np.bincount(y[test])))

=== Stratified K-Fold! ===
train [N neg, N Pos]:  [30  3]   |   test [N neg, N Pos]: [15  2]
train [N neg, N Pos]:  [30  3]   |   test [N neg, N Pos]: [15  2]
train [N neg, N Pos]:  [30  4]   |   test [N neg, N Pos]: [15  1]
=== UnStratified K-Fold! ===
train [N neg, N Pos]:   |   test [N neg, N Pos]:  [28  5]
train [N neg, N Pos]:   |   test [N neg, N Pos]:  [28  5]
train [N neg, N Pos]:   |   test [N neg, N Pos]:  [34]


### Function: `StratifiedShuffleSplit()`
StratifiedShuffleSplit is a variation of `ShuffleSplit`, which returns stratified splits, i.e which creates splits by preserving the same percentage for each target class as in the complete set.


## 3.1.2.3. Cross-validation iterators for grouped data.

**The i.i.d. assumption is broken if the underlying generative process yield groups of dependent samples.**

Such a grouping of data is domain specific. An example would be when there is medical data collected from multiple patients, with multiple samples taken from each patient. And such data is likely to be dependent on the individual group. In our example, the patient id for each sample will be its group identifier.

In this case we would like to know if a model trained on a particular set of groups generalizes well to the unseen groups. To measure this, we need to ensure that all the samples in the validation fold come from groups that are not represented at all in the paired training fold.

The following cross-validation splitters can be used to do that. The grouping identifier for the samples is specified via the groups parameter.

### Function : `GroupKFold()`
GroupKFold is a variation of k-fold which ensures that the same group is not represented in both testing and training sets. For example if the data is obtained from different subjects with several samples per-subject and if the model is flexible enough to learn from highly person specific features it could fail to generalize to new subjects. GroupKFold makes it possible to detect this kind of overfitting situations.

Imagine you have three subjects, each with an associated number from 1 to 3:

In [45]:
from sklearn.model_selection import GroupKFold

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]

gkf = GroupKFold(n_splits=3)
for train, test in gkf.split(X, y, groups=groups):
    print("%s %s" % (train, test))


[0 1 2 3 4 5] [6 7 8 9]
[0 1 2 6 7 8 9] [3 4 5]
[3 4 5 6 7 8 9] [0 1 2]


Each subject is in a different testing fold, and the same subject is never in both testing and training. Notice that the folds do not have exactly the same size due to the imbalance in the data.

Here is a visualization of the cross-validation behavior.

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_007.png' width=50%/>

### Function: `StratifiedGroupKFold()`
StratifiedGroupKFold is a cross-validation scheme that combines both StratifiedKFold and GroupKFold. The idea is to try to preserve the distribution of classes in each split while keeping each group within a single split. That might be useful when you have an unbalanced dataset so that using just GroupKFold might produce skewed splits.

In [47]:
from sklearn.model_selection import StratifiedGroupKFold
X = list(range(18))
y = [1] * 6 + [0] * 12
groups = [1, 2, 3, 3, 4, 4, 1, 1, 2, 2, 3, 4, 5, 5, 5, 6, 6, 6]
sgkf = StratifiedGroupKFold(n_splits=3)
for train, test in sgkf.split(X, y, groups=groups):
    print("%s %s" % (train, test))


[ 0  2  3  4  5  6  7 10 11 15 16 17] [ 1  8  9 12 13 14]
[ 0  1  4  5  6  7  8  9 11 12 13 14] [ 2  3 10 15 16 17]
[ 1  2  3  8  9 10 12 13 14 15 16 17] [ 0  4  5  6  7 11]
